In [3]:
!pip install ultralytics

In [10]:
!wget http://images.cocodataset.org/zips/val2017.zip
!unzip /content/val2017.zip.1

Streaming output truncated to the last 5000 lines.
 extracting: val2017/000000212226.jpg  
 extracting: val2017/000000231527.jpg  
 extracting: val2017/000000578922.jpg  
 extracting: val2017/000000062808.jpg  
 extracting: val2017/000000119038.jpg  
 extracting: val2017/000000114871.jpg  
 extracting: val2017/000000463918.jpg  
 extracting: val2017/000000365745.jpg  
 extracting: val2017/000000320425.jpg  
 extracting: val2017/000000481404.jpg  
 extracting: val2017/000000314294.jpg  
 extracting: val2017/000000335328.jpg  
 extracting: val2017/000000513688.jpg  
 extracting: val2017/000000158548.jpg  
 extracting: val2017/000000132116.jpg  
 extracting: val2017/000000415238.jpg  
 extracting: val2017/000000321333.jpg  
 extracting: val2017/000000081738.jpg  
 extracting: val2017/000000577584.jpg  
 extracting: val2017/000000346905.jpg  
 extracting: val2017/000000433980.jpg  
 extracting: val2017/000000228144.jpg  
 extracting: val2017/000000041872.jpg  
 extracting: val2017/00000011

Konfigurasi Dataset (mengganti format coco ke format yolo)

In [11]:
from pycocotools.coco import COCO
import os
import cv2

# Paths
coco_annotation_file = "/content/instances_val2017.json"
image_dir = "/content/val2017"
output_dir = "/content/yolo_format"

# Initialize COCO
coco = COCO(coco_annotation_file)

# Create output directories
os.makedirs(os.path.join(output_dir, "labels"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)

# Load categories
categories = coco.loadCats(coco.getCatIds())
category_mapping = {cat['id']: idx for idx, cat in enumerate(categories)}

# Process annotations
for image_info in coco.loadImgs(coco.getImgIds()):
    image_id = image_info['id']
    image_file = os.path.join(image_dir, image_info['file_name'])
    output_label_file = os.path.join(output_dir, "labels", f"{os.path.splitext(image_info['file_name'])[0]}.txt")

    # Write image to new directory
    if os.path.exists(image_file):
        os.rename(image_file, os.path.join(output_dir, "images", image_info['file_name']))

    # Write annotations
    with open(output_label_file, 'w') as f:
        for annotation in coco.loadAnns(coco.getAnnIds(imgIds=image_id)):
            category_id = annotation['category_id']
            bbox = annotation['bbox']  # [x, y, width, height]

            # Convert bbox to YOLO format (x_center, y_center, width, height)
            x_center = (bbox[0] + bbox[2] / 2) / image_info['width']
            y_center = (bbox[1] + bbox[3] / 2) / image_info['height']
            width = bbox[2] / image_info['width']
            height = bbox[3] / image_info['height']

            # Write to label file
            f.write(f"{category_mapping[category_id]} {x_center} {y_center} {width} {height}\n")


loading annotations into memory...
Done (t=0.59s)
creating index...
index created!


In [12]:
# Path to your COCO annotation file
annotation_file = "/content/instances_val2017.json"

# Load COCO annotations
coco = COCO(annotation_file)

# Get all category IDs and names
categories = coco.loadCats(coco.getCatIds())
class_names = [cat['name'] for cat in categories]

# Display class names and count
print(f"Number of classes: {len(class_names)}")
print("Classes:", class_names)


loading annotations into memory...
Done (t=0.80s)
creating index...
index created!
Number of classes: 80
Classes: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


Splitting Dataset (untuk training dan validation)

In [22]:
import os
import random
import shutil

# Paths to your dataset
images_path = "/content/yolo_format/images"
labels_path = "/content/yolo_format/labels"

# Output directories
train_images_path = "/content/training/train_images"
train_labels_path = "/content/training/train_labels"
val_images_path = "/content/training/val_images"
val_labels_path = "/content/training/val_labels"

# Create directories if they don't exist
os.makedirs(train_images_path, exist_ok=True)
os.makedirs(train_labels_path, exist_ok=True)
os.makedirs(val_images_path, exist_ok=True)
os.makedirs(val_labels_path, exist_ok=True)

# List all image files
image_files = [f for f in os.listdir(images_path) if f.endswith(".jpg")]

# Shuffle and split
random.shuffle(image_files)
split_ratio = 0.9
train_count = int(len(image_files) * split_ratio)

train_files = image_files[:train_count]
val_files = image_files[train_count:]

# Move files to train and val folders
for file in train_files:
    shutil.move(os.path.join(images_path, file), os.path.join(train_images_path, file))
    shutil.move(os.path.join(labels_path, file.replace(".jpg", ".txt")), os.path.join(train_labels_path, file.replace(".jpg", ".txt")))

for file in val_files:
    shutil.move(os.path.join(images_path, file), os.path.join(val_images_path, file))
    shutil.move(os.path.join(labels_path, file.replace(".jpg", ".txt")), os.path.join(val_labels_path, file.replace(".jpg", ".txt")))

print(f"Training images: {len(train_images_path)}")
print(f"Validation images: {len(val_images_path)}")

Training images: 30
Validation images: 28


In [23]:
import os
import shutil

# Define paths (assuming these variables are already defined)
train_images_path = "/content/training/train_images"
train_labels_path = "/content/training/train_labels"
val_images_path = "/content/training/val_images"
val_labels_path = "/content/training/val_labels"


# Function to move labels into image directories
def move_labels_to_images(images_path, labels_path):
  for filename in os.listdir(images_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust extensions as needed
      image_name = filename[:-4]  # Remove the extension
      label_file = f"{image_name}.txt"
      label_src = os.path.join(labels_path, label_file)
      label_dest = os.path.join(images_path, label_file)

      if os.path.exists(label_src):
          shutil.move(label_src, label_dest)
          print(f"Moved {label_file} to {images_path}")
      else:
          print(f"Label file not found for {filename}")

# Move labels for training and validation sets
move_labels_to_images(train_images_path, train_labels_path)
move_labels_to_images(val_images_path, val_labels_path)

print("Label moving complete.")

Streaming output truncated to the last 5000 lines.
Moved 000000212573.txt to /content/training/train_images
Moved 000000289594.txt to /content/training/train_images
Moved 000000480212.txt to /content/training/train_images
Moved 000000286553.txt to /content/training/train_images
Moved 000000192716.txt to /content/training/train_images
Moved 000000229216.txt to /content/training/train_images
Moved 000000302165.txt to /content/training/train_images
Moved 000000341469.txt to /content/training/train_images
Moved 000000499181.txt to /content/training/train_images
Moved 000000533493.txt to /content/training/train_images
Moved 000000524850.txt to /content/training/train_images
Moved 000000400803.txt to /content/training/train_images
Moved 000000168337.txt to /content/training/train_images
Moved 000000411754.txt to /content/training/train_images
Moved 000000581100.txt to /content/training/train_images
Moved 000000547336.txt to /content/training/train_images
Moved 000000343315.txt to /content/tr

In [26]:
# jumlah dataset

print(f"Training images: {len(os.listdir(train_images_path))}")
print(f"Validation images: {len(os.listdir(val_images_path))}")

Training images: 9000
Validation images: 1000


Konfigurasi .yaml file

In [27]:
with open('/content/coco_val2017.yaml', 'w') as file:
    file.write("""
train: /content/training/train_images
val: /content/training/val_images

nc: 80
names:
  - person
  - bicycle
  - car
  - motorcycle
  - airplane
  - bus
  - train
  - truck
  - boat
  - traffic light
  - fire hydrant
  - stop sign
  - parking meter
  - bench
  - bird
  - cat
  - dog
  - horse
  - sheep
  - cow
  - elephant
  - bear
  - zebra
  - giraffe
  - backpack
  - umbrella
  - handbag
  - tie
  - suitcase
  - frisbee
  - skis
  - snowboard
  - sports ball
  - kite
  - baseball bat
  - baseball glove
  - skateboard
  - surfboard
  - tennis racket
  - bottle
  - wine glass
  - cup
  - fork
  - knife
  - spoon
  - bowl
  - banana
  - apple
  - sandwich
  - orange
  - broccoli
  - carrot
  - hot dog
  - pizza
  - donut
  - cake
  - chair
  - couch
  - potted plant
  - bed
  - dining table
  - toilet
  - TV
  - laptop
  - mouse
  - remote
  - keyboard
  - cell phone
  - microwave
  - oven
  - toaster
  - sink
  - refrigerator
  - book
  - clock
  - vase
  - scissors
  - teddy bear
  - hair drier
  - toothbrush
""")

Training Model

In [30]:
from ultralytics import YOLO

# Load the pretrained YOLOv8 Nano model
model = YOLO('yolov8n.pt')

# Train the model
model.train(
    data='/content/coco_val2017.yaml',  # Path to your YAML file
    epochs=20,                         # Number of epochs
    imgsz=640,                         # Image size
    batch=16,                          # Batch size
    lr0=0.01,                          # Initial learning rate
    workers=4,                         # Number of CPU workers for data loading
    project='yolov8_training',         # Save results in this folder
    name='yolov8n_20epochs'            # Name for this training run
)

# Evaluate the model
results = model.val()
print(results)

Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/coco_val2017.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=yolov8_training, name=yolov8n_20epochs, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show

train: Scanning /content/training/train_images.cache... 4500 images, 43 backgrounds, 0 corrupt: 100%|██████████| 4500/4500 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/training/val_images.cache... 500 images, 5 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]


Plotting labels to yolov8_training/yolov8n_20epochs/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to yolov8_training/yolov8n_20epochs
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        500       3679      0.597      0.443      0.496      0.356

20 epochs completed in 0.644 hours.
Optimizer stripped from yolov8_training/yolov8n_20epochs/weights/last.pt, 6.5MB
Optimizer stripped from yolov8_training/yolov8n_20epochs/weights/best.pt, 6.5MB

Validating yolov8_training/yolov8n_20epochs/weights/best.pt...
Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:11<00:00,  1.37it/s]


                   all        500       3679      0.622      0.456      0.514      0.375
                person        266       1105      0.791      0.671      0.765      0.526
               bicycle         12         16          1      0.428      0.613      0.302
                   car         51        189      0.642      0.444      0.509      0.306
            motorcycle         12         46      0.665       0.63      0.681      0.405
              airplane         11         15      0.685      0.733      0.831      0.658
                   bus         17         24      0.956      0.667      0.826       0.68
                 train         14         15       0.88        0.8      0.888      0.758
                 truck         23         28      0.516      0.429      0.425      0.302
                  boat         19         71      0.598      0.254      0.328      0.186
         traffic light         14         47      0.665      0.255      0.271      0.135
          fire hydran

val: Scanning /content/training/val_images.cache... 500 images, 5 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:06<00:00,  5.02it/s]


                   all        500       3679      0.625      0.457      0.516      0.377
                person        266       1105      0.792       0.67      0.765      0.527
               bicycle         12         16          1      0.428      0.615      0.305
                   car         51        189      0.651      0.444      0.517      0.307
            motorcycle         12         46      0.644      0.652      0.681      0.412
              airplane         11         15      0.644      0.733      0.825      0.652
                   bus         17         24      0.957      0.667       0.83      0.672
                 train         14         15      0.879        0.8      0.893      0.743
                 truck         23         28      0.516      0.429      0.427      0.302
                  boat         19         71       0.62      0.254      0.329      0.185
         traffic light         14         47      0.705      0.255      0.277      0.132
          fire hydran